In [1]:
from sklearn.datasets import fetch_california_housing

def linear_model_demo():
    """
    线性回归直接预测房子价格
    :return: None
    """
    # 获取数据
    # 修改说明：这里已将 data_home 路径修改为 'day14/data'
    fe_cal = fetch_california_housing(data_home='day14/data')

    print("获取特征值")
    print(fe_cal.data.shape)
    print('-' * 50)
    print(fe_cal.data[0])

    print("目标值")
    print(fe_cal.target) # 单位是10万美金

    # 注意：DESCR 输出可能会比较长，根据需要可以注释掉
    # print(fe_cal.DESCR)

    print('-' * 50)
    print(fe_cal.feature_names) # 特征列的名字

# 程序入口
if __name__ == "__main__":
    linear_model_demo()

获取特征值
(20640, 8)
--------------------------------------------------
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
目标值
[4.526 3.585 3.521 ... 0.923 0.847 0.894]
--------------------------------------------------
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def linear_model_demo():
    """
    线性回归：数据获取、分割与标准化
    :return: None
    """
    # 1. 获取数据
    fe_cal = fetch_california_housing(data_home='day14/data')

    print("目标值形状:", fe_cal.target.shape)

    # 2. 分割数据集到训练集和测试集
    # test_size=0.25 表示25%的数据用于测试，75%用于训练
    # random_state=1 保证每次运行分割结果一致
    x_train, x_test, y_train, y_test = train_test_split(
        fe_cal.data, fe_cal.target, test_size=0.25, random_state=1
    )

    print("训练集特征值形状:", x_train.shape) # 应该是 (15480, 8)

    # 3. 特征值标准化处理
    # 实例化标准化API
    std_x = StandardScaler()

    # 对训练集进行标准化 (fit_transform = 计算平均值和方差 + 转换)
    x_train = std_x.fit_transform(x_train)

    # 对测试集进行标准化 (transform = 使用训练集的平均值和方差进行转换)
    # 注意：这里不能用 fit_transform，否则会造成数据泄露
    x_test = std_x.transform(x_test)

    # 4. 目标值标准化处理 (根据截图，这部分暂时被注释或者是可选的)
    # 如果后续使用 SGDRegressor (梯度下降)，通常建议对目标值也进行标准化
    # 如果使用 LinearRegression (正规方程)，通常不需要对目标值标准化

    # std_y = StandardScaler()

    # reshape(-1, 1) 的作用是将一维数组 [1, 2, 3] 转换为二维的列向量 [[1], [2], [3]]
    # 因为 StandardScaler 要求输入必须是二维的

    # y_train = std_y.fit_transform(y_train.reshape(-1, 1))
    # y_test = std_y.transform(y_test.reshape(-1, 1))

    print("数据处理完成")

# 程序入口
if __name__ == "__main__":
    linear_model_demo()

目标值形状: (20640,)
训练集特征值形状: (15480, 8)
数据处理完成


In [2]:
import os
import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def linear_model_demo():
    """
    线性回归：正规方程预测房子价格
    :return: None
    """
    # 1. 获取数据
    fe_cal = fetch_california_housing(data_home='day14/data')

    # 打印目标值形状 (对应第一张图 [14])
    print("目标值形状:", fe_cal.target.shape)

    # 2. 分割数据集到训练集和测试集 (对应第一张图 [15])
    x_train, x_test, y_train, y_test = train_test_split(
        fe_cal.data, fe_cal.target, test_size=0.25, random_state=1
    )

    print("训练集特征值形状:", x_train.shape)

    # 3. 进行标准化处理 (对应第一、二张图)
    # 特征值和目标值是都必须进行标准化处理，实例化两个标准化API
    std_x = StandardScaler()

    x_train = std_x.fit_transform(x_train) # 训练集标准化
    x_test = std_x.transform(x_test)       # 测试集标准化

    # 目标值进行了标准化，暂时没有对目标值进行标准化处理 (截图中的注释代码)
    std_y = StandardScaler()

    temp = y_train.reshape(-1, 1) # -1代表把剩余的元素都堆到哪一维

    # 标签进行标准化
    # 目标值是一维的，这里需要传进去2维的
    y_train = std_y.fit_transform(y_train.reshape(-1, 1))#这样会变成n行1列
    print(y_train.shape)
    y_test = std_y.transform(y_test.reshape(-1, 1))
    print(y_test.shape)

    # 4. estimator 预测 (对应第三张图 [17])
    # 正规方程求解方式预测结果，正规方程进行线性回归
    lr = LinearRegression()

    # fit 是耗时的
    lr.fit(x_train, y_train)

    # 回归系数可以看特征与目标值之间的相关性
    print('回归系数', lr.coef_)

    y_predict = lr.predict(x_test)

    # 预测测试集的房子价格，通过inverse得到真正的房子价格 (注释代码)
    # y_lr_predict = std_y.inverse_transform(y_predict)

    # 5. 保存训练好的模型 (对应第三张图)
    # 模型中保存的是w的值，也保存了模型结构
    # 保存模型放在fit之后即可

    # 确保目录存在（为了代码健壮性，我添加了目录检查，否则直接unlink可能会报错）
    if not os.path.exists('./tmp'):
        os.makedirs('./tmp')

    # 删除之前的模型文件 (如果有的话)
    if os.path.exists('./tmp/test.pkl'):
        os.unlink('./tmp/test.pkl')

    joblib.dump(lr, "./tmp/test.pkl")

    print("正规方程测试集里面每个房子的预测价格: ", y_predict[0:10])

    # 6. 模型评估 (对应第三张图)
    # 下面是求测试集的损失，用均方误差，公式是(y_test - y_predict)^2 / n
    print("正规方程的均方误差: ", mean_squared_error(y_test, y_predict))

# 程序入口
if __name__ == "__main__":
    linear_model_demo()

目标值形状: (20640,)
训练集特征值形状: (15480, 8)
(15480, 1)
(5160, 1)
回归系数 [[ 0.71942632  0.10518431 -0.23147194  0.26802332 -0.00448136 -0.03495117
  -0.7849086  -0.76307353]]
正规方程测试集里面每个房子的预测价格:  [[ 0.039975  ]
 [-0.9856667 ]
 [ 0.54595901]
 [-0.31917221]
 [ 0.65037085]
 [ 1.23359413]
 [ 0.81054876]
 [-0.38917515]
 [-0.28938242]
 [-0.05080248]]
正规方程的均方误差:  0.40082431136214186


In [4]:
import os
import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def linear_model_demo():
    """
    线性回归：正规方程预测房子价格
    :return: None
    """
    # 1. 获取数据
    fe_cal = fetch_california_housing(data_home='day14/data')

    # [14] 打印目标值形状
    print(fe_cal.target.shape)

    # [15] 分割数据集到训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(
        fe_cal.data, fe_cal.target, test_size=0.25, random_state=1
    )

    print(x_train.shape)

    # 进行标准化处理(?) 目标值处理?
    # 特征值和目标值是都必须进行标准化处理，实例化两个标准化API
    std_x = StandardScaler()

    x_train = std_x.fit_transform(x_train) # 训练集标准化
    x_test = std_x.transform(x_test)       # 测试集标准化

    # 目标值进行了标准化，暂时没有对目标值进行标准化处理 (截图中的注释代码)
    std_y = StandardScaler()

    # temp = y_train.reshape(-1, 1) # -1代表把剩余的元素都堆到哪一维

    # # 标签进行标准化
    # # 目标值是一维的，这里需要传进去2维的
    y_train = std_y.fit_transform(y_train.reshape(-1, 1))
    print(y_train.shape)
    y_test = std_y.transform(y_test.reshape(-1, 1))
    print(y_test.shape)

    # [17] estimator 预测
    # 正规方程求解方式预测结果，正规方程进行线性回归
    lr = LinearRegression()

    # fit 是耗时的
    lr.fit(x_train, y_train)

    # 回归系数可以看特征与目标值之间的相关性
    print('回归系数', lr.coef_)

    y_predict = lr.predict(x_test)

    # 预测测试集的房子价格，通过inverse得到真正的房子价格 (注释代码)
    y_lr_predict = std_y.inverse_transform(y_predict)

    # 保存训练好的模型，模型中保存的是w的值，也保存了模型结构
    # 保存模型放在fit之后即可

    # 注意：如果文件不存在，os.unlink可能会报错，建议加个判断或者忽略错误
    if os.path.exists('./tmp/test.pkl'):
        os.unlink('./tmp/test.pkl') # 删除之前的模型文件

    # 确保存储目录存在
    if not os.path.exists('./tmp'):
        os.makedirs('./tmp')

    joblib.dump(lr, "./tmp/test.pkl")

    print("正规方程测试集里面每个房子的预测价格: ", y_predict[0:10])

    # 下面是求测试集的损失，用均方误差，公式是(y_test - y_predict)^2 / n
    print("正规方程的均方误差: ", mean_squared_error(y_test, y_predict))

# 程序入口
if __name__ == "__main__":
    linear_model_demo()

(20640,)
(15480, 8)
(15480, 1)
(5160, 1)
回归系数 [[ 0.71942632  0.10518431 -0.23147194  0.26802332 -0.00448136 -0.03495117
  -0.7849086  -0.76307353]]
正规方程测试集里面每个房子的预测价格:  [[ 0.039975  ]
 [-0.9856667 ]
 [ 0.54595901]
 [-0.31917221]
 [ 0.65037085]
 [ 1.23359413]
 [ 0.81054876]
 [-0.38917515]
 [-0.28938242]
 [-0.05080248]]
正规方程的均方误差:  0.40082431136214186


下面是做了标准化的正规方程

In [6]:
import os
import joblib
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def linear_model_demo():
    """
    线性回归：正规方程预测房子价格（包含模型保存与加载）
    :return: None
    """
    # ------------------- 1. 数据准备与训练 -------------------
    # 获取数据
    fe_cal = fetch_california_housing(data_home='day14/data')

    # [14] 打印目标值形状
    # print(fe_cal.target.shape) # (20640,)

    # [15] 分割数据集到训练集和测试集
    x_train, x_test, y_train, y_test = train_test_split(
        fe_cal.data, fe_cal.target, test_size=0.25, random_state=1
    )

    # print(x_train.shape)

    # 特征值和目标值都必须进行标准化处理，实例化标准化API
    std_x = StandardScaler()

    # 训练集标准化
    x_train = std_x.fit_transform(x_train)
    # 测试集标准化
    x_test = std_x.transform(x_test)

    # [目标值标准化逻辑 - 截图显示已被注释]
    # std_y = StandardScaler()
    # temp = y_train.reshape(-1, 1) # -1代表把剩余的元素都堆到哪一维
    # y_train = std_y.fit_transform(y_train.reshape(-1, 1))
    # y_test = std_y.transform(y_test.reshape(-1, 1))

    # [17] estimator 预测
    # 正规方程求解方式预测结果，正规方程进行线性回归
    lr = LinearRegression()

    # fit 是耗时的
    lr.fit(x_train, y_train)

    # 回归系数可以看特征与目标值之间的相关性
    print('回归系数', lr.coef_)

    y_predict = lr.predict(x_test)

    # [反标准化逻辑 - 截图显示已被注释]
    # y_lr_predict = std_y.inverse_transform(y_predict)

    # ------------------- 2. 模型保存 -------------------
    # 保存训练好的模型，模型中保存的是w的值，也保存了模型结构
    # 保存模型放在fit之后即可

    # 确保目录存在
    if not os.path.exists('./tmp'):
        os.makedirs('./tmp')

    # 删除之前的模型文件
    if os.path.exists('./tmp/test.pkl'):
        os.unlink('./tmp/test.pkl')

    joblib.dump(lr, "./tmp/test.pkl")

    print("正规方程测试集里面每个房子的预测价格: ", y_predict[0:10])

    # 下面是求测试集的损失，用均方误差
    print("正规方程的均方误差: ", mean_squared_error(y_test, y_predict))

    print("=" * 30)

    # ------------------- 3. 模型加载 (对应最后一张图) -------------------
    # 模拟上线时加载模型
    model = joblib.load("./tmp/test.pkl")

    # 预测
    # 注意：这里的 x_test 已经是经过 std_x 标准化过的数据了
    y_predict_load = model.predict(x_test)

    print("保存的模型预测的结果: ", y_predict_load[0:10])
    print("正规方程的均方误差: ", mean_squared_error(y_test, y_predict_load))

    # [注释代码：如果目标值进行了标准化，这里需要inverse_transform]
    # print("保存的模型预测的结果: ", std_y.inverse_transform(y_predict_load)[0:10])
    # print("正规方程inverse后的均方误差: ", mean_squared_error(std_y.inverse_transform(y_test),
    #                                                  std_y.inverse_transform(y_predict_load)))

# 程序入口
if __name__ == "__main__":
    linear_model_demo()

回归系数 [ 0.83167028  0.12159502 -0.26758589  0.30983997 -0.00518054 -0.04040421
 -0.90736902 -0.88212727]
正规方程测试集里面每个房子的预测价格:  [2.12391852 0.93825754 2.7088455  1.70873764 2.82954754 3.50376456
 3.0147162  1.62781292 1.74317518 2.01897806]
正规方程的均方误差:  0.5356532845422556
保存的模型预测的结果:  [2.12391852 0.93825754 2.7088455  1.70873764 2.82954754 3.50376456
 3.0147162  1.62781292 1.74317518 2.01897806]
正规方程的均方误差:  0.5356532845422556
